###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2022  by D. Koehn, T. Meier and J. Stampa, notebook style sheet by L.A. Barba, N.C. Clementi

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# Digitale Signalbearbeitung in der Geophysik 

## Kapitel 4 Vorbereitende Datenbearbeitung

### 4.4 Einfache Transformationen

Oft sind bereits einfache Transformationen hilfreich, um das gewünschte Signal besser sichtbar zu machen oder Fehler zu beseitigen. Interessieren vor allem die hohen Frequenzen und die tiefen Frequenzen stören eher, kann eine zeitliche Differentiation der Wertereihe erfolgen - wenn nötig auch mehrmals. Eine Integration bzgl. der Zeit hilft dagegen, wenn eher die tiefen Frequenzen interessieren. Allerdings kann durch Integration auch langperiodisches Rauschen verstärkt werden, weshalb die Kombination mit einer Filterung empfehlenswert ist. Ein weiteres Beispiel ist die Amplitudenkorrektur z.B. bei reflexionsseismischen Messungen: die Amplitudenabnahme mit der Zeit wird korrigiert, um schwache spätere Einsätze besser sichtbar zu machen und künstlich Stationarität der Wertereihe bzgl. der Amplitude herbei zu führen.

Ist die Amplitude nur bis auf einen unbekannten Faktor bestimmt, z.B. aufgrund unbekannter Ankopplung eines Geophons an den Untergrund, wird die Zeitreihe auf das Maximum der Wertereihe normiert. Es wird dann die Wellenform aber nicht ihr absoluter Wert interpretiert. Das ist bei vielen Anwendungen ausreichend, da oft nur Laufzeiten oder die Phase des Signals aber nicht die Amplitude ausgewertet werden.    

Um kleinere Signale sichtbar zu machen, kann eine Logarithmierung der Wertereihe erfolgen:

\begin{equation}
\tilde{x}_i = \ln {x_i}. \tag{4.4}
\end{equation}

Schauen wir uns das an einem Beispiel genauer an. Wie üblich importieren wir als erstes einige Python Bibliotheken.

In [2]:
# Importiere Python Bibliotheken 
# ------------------------------
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

Im nächsten Schritt verwenden wir die Sinusfunktion von Übungsaufgabe 3.4 und addieren einige amplitudenstarke Spikes, deren Amplituden wir interaktiv ändern können. Über einen  Schalter läßt sich die Logarithmierung der Zeitreihe aktivieren. 

In [3]:
def x_log(spike_amp,logx):
    
    # Define parameters
    T = 25.                  # period 1 [s]
    dt = .1                  # time sampling [s]
    L = 1000.                # length of the time series [s]

    # Define sine function ...
    omega = 2. * np.pi / T   # compute circular frequency from period
    t = np.arange(0,L+dt,dt) # compute time vector
    x = np.sin(omega*t)  # compute sine wave
    
    # Add spike to the time series
    N = len(x)
    x[N//2] = spike_amp
    
    if(logx==True):
        
        # if x > 0
        x = np.log(x)
    
        # if x <=0
        #C = 1 / np.log(10)
        #x = np.sign(x) * np.log(1+np.abs(x/C))
    
    # Initialize Plots
    plt.figure(figsize=(10,5))
    
    # Plot time series
    plt.plot(t, x, 'b')
    plt.xlabel('Time (s)')
    plt.ylabel('f(t) (s)')
    
    if(logx==True):
        plt.ylabel('log(f(t)) (s)')
        
    plt.title('Time Series f(t) = Sin(t) + Spike(t)')

Als nächstes können wir mit der erzeugten Zeitreihe und der Logarithmierung herumspielen ..

In [4]:
interactive_plot = interactive(x_log, spike_amp=(0., 1000.,50), logx=False)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=500.0, description='spike_amp', max=1000.0, step=50.0), Checkbox(value…

Allerdings ist das eine nicht-lineare Transformation. Eventuell vorhandene lineare Zusammenhänge werden dadurch zerstört. Eine weitere digitale Bearbeitung der Daten ist dann meistens nicht mehr angebracht. Eine andere nicht-lineare Transformation, die sehr flexibel eingesetzt werden kann, um entweder schwache Signale sichtbar zu machen oder störendes Hintergrundrauschen zu unterdrücken ist:

\begin{equation}
\tilde{x}_i = sgn(x_i) |x_i|^{n}. \tag{4.5}
\end{equation} 

Für $n = 1$ ergibt sich keine Änderung, für $n<1$ werden dagegen kleinere Amplituden verstärkt. Diese Transformation eignet sich aufgrund ihrer Flexibilität vor allem für die Darstellung von Wertereihen mit großem Dynamikumfang. Für $n>1$ werden schwächere Signale unterdrückt.

Testen wir Transformation (4.5) an der Zeitreihe mit der Sinusschwingung und dem Spike.

In [5]:
def sgnxxn(n,sgnxxn):
    
    # Define parameters
    T = 25.                  # period 1 [s]
    dt = .1                  # time sampling [s]
    L = 1000.                # length of the time series [s]
    spike_amp = 1000.        # spike amplitude

    # Define sine function ...
    omega = 2. * np.pi / T   # compute circular frequency from period
    t = np.arange(0,L+dt,dt) # compute time vector
    x = np.sin(omega*t)  # compute sine wave
    
    # Add spike to the time series
    N = len(x)
    x[N//2] = spike_amp
    
    if(sgnxxn==True):
        x = np.sign(x) * np.abs(x)**n
    
    # Initialize Plots
    plt.figure(figsize=(10,5))
    
    # Plot time series
    plt.plot(t, x, 'b')
    plt.xlabel('Time (s)')
    plt.ylabel('f(t) (s)')
    
    if(sgnxxn==True):
        plt.ylabel('sgn(f(t))abs(f(t))**n (s)')
        
    plt.title('Time Series f(t) = Sin(t) + Spike(t)')

In [6]:
interactive_plot = interactive(sgnxxn, n=(0., 2., .1), sgnxxn=False)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=1.0, description='n', max=2.0), Checkbox(value=False, description='sgn…

Mitunter wird auch die Wertereihe zu einem Binärsignal reduziert, um Stationarität zu erzeugen und große Amplitudenunterschiede zu beseitigen:

\begin{equation}
\tilde{x}_i = \begin{cases} 1,& x_i \geq 0 \\ -1,& x_i < 0. \end{cases} \tag{4.6}
\end{equation}

Erstaunlicherweise kann diese stark nicht-lineare Transformation zur Herstellung der [Stationarität](https://en.wikipedia.org/wiki/Stationary_process) für die Bestimmung der Greensfunktion aus dem Hintergrundrauschen erfolgreich eingesetzt werden.

Um zu testen, ob eine Zeitreihe stationär ist, kann man den [Dickey-Fuller Test](https://pythondata.com/stationary-data-tests-for-time-series-forecasting/) verwenden. Dazu müssen wir nur die passende Python Funktion importieren ...

In [7]:
from statsmodels.tsa.stattools import adfuller

Um zu testen, ob die Transformation $(4.6)$ Stationarität erzeugt, addieren wir auf unsere Sinusfunktion mit der Amplitude $a$ einen linearen Trend mit der Steigung $b$

In [8]:
def sgnx(a,b,sgnx):
    
    # Define parameters
    T = 25.                  # period 1 [s]
    dt = .1                  # time sampling [s]
    L = 1000.                # length of the time series [s]

    # Define sine function ...
    omega = 2. * np.pi / T   # compute circular frequency from period
    t = np.arange(0,L+dt,dt) # compute time vector
    x = a*np.sin(omega*t)  # compute sine wave
    
    # Add linear trend to the time series
    xlin = b * t
    x = x + xlin
        
    if(sgnx==True):
        x = np.sign(x)
    
    # Apply Dickey-Fuller test to x
    res = adfuller(x)
    
    # Initialize Plots
    plt.figure(figsize=(10,5))
    
    # Plot time series
    plt.plot(t, x, 'b')
    plt.xlabel('Time (s)')
    plt.ylabel('f(t) (s)')
    
    if(sgnx==True):
        plt.ylabel('sgn(f(t)) (s)')
    
    title = 'Time Series f(t) = a*Sin(t) + b*t, Dickey Fuller Test p-value = ' + str(res[1])
    
    plt.title(title)

Genaugenommen, haben wir in der Funktion `sgnx` nicht Gl. (4.6) sondern die `NumPy` Funktion [sign](https://numpy.org/doc/stable/reference/generated/numpy.sign.html) implementiert.

In [9]:
interactive_plot = interactive(sgnx, a=(0., 10., .1), b=(-.05, .05, .005),  sgnx=False)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=5.0, description='a', max=10.0), FloatSlider(value=0.0, description='b…

Für die Sinusfunktion ohne linearen Trend ($b=0$) liefert der Dickey Fuller Test einen p-Wert von 0, womit wir eine stationäre Zeitreihe haben. Selbst bei kleinen Variationen von $b \ne 0$, ergeben sich jedoch p-Werte nahe bei 1, womit die Zeitreihe nicht mehr stationär ist. 

Durch Anwendung der Transformation $sgn(x)$ springt der p-Wert sofort wieder auf Null, womit gezeigt wurde, daß die Signum-Transformation Stationarität herstellen kann. In diesem Fall geht dadurch jedoch ein großer Teil an Informationen verloren. Ein besserer Ansatz ist die Anwendung einer Trendkorrektur, die in Abschnitt 4.5 genauer diskutiert wird.

Für die Detektion eines Signals kann die folgende Transformation angewendet werden:

\begin{equation}
\tilde{x}_i = \begin{cases} 0,& x_i < \mbox{Schwellwert xc} \\ 1,& x_i \geq \mbox{Schwellwert xc}. \end{cases} \tag{4.7}
\end{equation}

Das Resultat ist gleich Eins, falls die Amplitude der Wertereihe über einem Schwellwert liegt und damit das Vorhandensein eines Signals anzeigt.

Testen wir das erneut für unsere Sinuszeitreihe, wobei wir einen Teil am Anfang der Zeitreihe isolieren, die Amplituden mit einer Gaussfunktion dämpfen und etwas zufallsverteilten Noise draufaddieren.

In [10]:
def sgndetect(xc,an,detect,damp,noise):
    
    # Define parameters
    T = 25.                  # period 1 [s]
    dt = .1                  # time sampling [s]
    L = 1000.                # length of the time series [s]

    # Define sine function ...
    omega = 2. * np.pi / T   # compute circular frequency from period
    t = np.arange(0,L+dt,dt) # compute time vector
    x = np.sin(omega*t)  # compute sine wave
    
    # Isolate a part of the time series 
    N = len(x)
    t1 = (int) (50 // dt)
    t2 = (int) (100 // dt)
    t3 = (int) (200 // dt)
    x[:t1] = 0.
    x[t2:t3] = 0.
    
    # Damp time series with Gaussian function before t3
    if(damp==True):
        a=1e-6
        xdamp = x * np.exp(-a*(t-t3)**2)
        x[:t3] = xdamp[:t3]
        
    # Add normal distributed noise to the time series
    if(noise==True):
        mu = 0.
        sigma = .1
        xnoise = np.random.normal(mu, sigma, N)
        x = x + an * xnoise
    
    if(detect==True):
        x[x>=xc] = 1.
        x[x<xc] = 0.
    
    # Initialize Plots
    plt.figure(figsize=(10,5))
    
    # Plot time series
    plt.plot(t, x, 'b')
    plt.xlabel('Time (s)')
    plt.ylabel('f(t) (s)')
    
    if(detect==True):
        plt.ylabel('sgn(f(t)) (s)')
    
    title = 'Time Series f(t) = Sin(t)'
    
    plt.title(title)

In [11]:
interactive_plot = interactive(sgndetect, xc=(0., .04, .005), an=(.0, .2, .01), detect=False, damp=False, noise=False)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.02, description='xc', max=0.04, step=0.005), FloatSlider(value=0.1, …

Mitunter reicht die Multiplikation der Wertereihe mit einem konstanten Faktor, um Fehler zu korrigieren. Die fehlerhaften Wertereihen müssen nur mit einem Korrekturfaktor $a$ multipliziert werden, der allerdings unbekannt ist. Um ihn zu bestimmen, wird ein Modell für die Wertereihe angenommen. Für die korrigierte Wertereihe $\hat{x}_i$ gilt danach:

\begin{equation}
\hat{x}_i = a \tilde{x}_i, \tag{4.8}
\label{eq:Faktor}
\end{equation}  

wobei $\tilde{x}_i$ die unkorrigierte, gemessene Wertereihe ist.

Wir wollen das ganze an einer Zeitreihe ähnlich dem Huddle Test untersuchen. Dazu addieren wir auf unsere Sinusfunktion mit einer Periode von $T_1 = 25 s$ eine zweite Sinusfunktion mit einer Periode von $T_2 = 500 s$.

In [12]:
def fscale(a):
    
    # Define parameters
    T1 = 25.                 # period 1 [s]
    T2 = 500.                 # period 2 [s]
    dt = .1                  # time sampling [s]
    L = 1000.                # length of the time series [s]

    # Define sine functions ...
    omega1 = 2. * np.pi / T1   # compute circular frequency from period 1
    omega2 = 2. * np.pi / T2   # compute circular frequency from period 2
    t = np.arange(0,L+dt,dt) # compute time vector
    
    x1 = np.sin(omega1*t)  # compute sine wave 1
    x2 = np.sin(omega2*t)  # compute sine wave 1
    
    # Add both sine functions
    x = x1 + x2
    
    # Define uncorrected time series
    ar = 1.27
    xb = x * a
    
    # Define corrected time series
    xh = x * ar
    
    # Initialize Plots
    plt.figure(figsize=(10,5))
    
    # Plot time series
    plt.plot(t, xb, 'b', label = r'uncorrected time series $\overline{x}$')
    plt.plot(t, xh, 'r', label = r'corrected time series $\hat{x}$')
    plt.xlabel('Time (s)')
    plt.ylabel('f(t) (s)')
    plt.legend()
    title = 'Time Series f(t) = Sin(t)'
    
    plt.title(title)

Durch Variation des Faktors $a$ können wir die unkorrigierte Zeitreihe $\overline{x}$ in die korrigierte Zeitreihe $\hat{x}$ transformieren. 

In [13]:
interactive_plot = interactive(fscale, a=(.0, 10., .1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=5.0, description='a', max=10.0), Output(layout=Layout(height='350px'))…

Nimmt man an, dass durch ein benachbartes Referenzseismometer $x_i$ gemessen worden ist und deswegen als bekannt angenommen werden kann, lässt sich der Korrekturfaktor $a$ mit Hilfe der Methode der kleinsten Quadrate bestimmen. Dabei wird die Fehlerquadratsumme

\begin{equation}
E = \sum_{i=1}^N (x_i - \hat{x}_i)^2 \tag{4.9}
\end{equation} 

minimiert. D.h. für den Korrekturfaktor soll gelten:

\begin{equation}
\frac{\partial E}{\partial a}\stackrel{!}{=}0. \tag{4.10}
\end{equation}

Diese Bedingung führt auf eine einfache Bestimmungsgleichung für $a$. Durch Multiplikation mit $a$ wird die korrigierte Wertereihe erhalten. 

Ähnlich kann man vorgehen, wenn in einer Wertereihe $x_i$ eine Störgröße vorhanden ist, die durch eine externe Messung $p_i$ identifiziert und gemessen werden kann. Im einfachsten Fall kann ein lineares Modell und zwar speziell  

\begin{equation}
\hat{x}_i=f(p_i) = ap_i \tag{4.11}
\label{eq:extern}
\end{equation}

angenommen werden. Dabei kann $x_i$ z.B. eine kontinuierliche Schweremessung und $p_i$ die Messung des Luftdrucks sein. $\hat{x}_i$ ist der Anteil der Schweremessung, der durch den Luftdruck hervorgerufen wird. Ist der Faktor $a$ bekannt, der beschreibt, wie stark die Störgröße in der gemessenen Wertereihe auftritt, kann die externe Messung der Störgröße $p_i$ mit $a$ multipliziert und von der gemessenen Wertereihe abgezogen werden, um die korrigierte Wertereihe zu erhalten. Oft ist das nicht der Fall und er muss mittels der Methode der kleinsten Quadrate bestimmt werden. Die Fehlerquadratsumme lautet in diesem Fall: 

\begin{equation}
E = \sum_{i=1}^N (x_i - a p_i)^2. \tag{4.12}
\end{equation} 

Ihre Minimierung führt auf eine Bestimmungsgleichung für $a$ ähnliche wie im vorangegangenen Beispiel.

## Zusammenfassung:

Mit Hilfe von einfachen Transformationen können ... 

- schwache Signale sichtbar gemacht

- die Stationarität von Zeitreihen hergestellt

- Signale detektiert 

- Amplitudenunterschiede korrigiert

werden.